In [2]:
%run runaway_functionsv2
%matplotlib qt


In [3]:
cluster_name = 'Berkeley_97'
cluster = Cluster(cluster_name)
# cluster.plot_search_region(pixels='800')


In [3]:
%%time
runCode(cluster_name)

=======================================Berkeley_97==================================================
All tables present (stars_in_region,stars_fromDR3,stars_fromDR3_dis,fs,nsfs)
Berkeley_97/Berkeley_97_runaways_all.tsv exists. Skipping traceback and temp estimates.
261 stars had been checked, 13 traced back to cluster
Berkeley_97
Av: 3.1
logage: 7.0
[Fe/H]: -0.1
Theoretical isochrone exists, reading it.
Berkeley_97/Berkeley_97_stars_in_region.tsv exists in Berkeley_97 with 446 stars
mean(v_pec):  2.7685305586193776
std(v_pec):  2.3812339786561485
v_pec_max:  10.292515324099009


v_pec,Temp. Est,µ_pec,_q_1,_r_1,RA_ICRS_1,rmRA,e_rmRA,rmDE,e_rmDE,DE_ICRS_1,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,FBP,e_FBP,BPmag,FRP,e_FRP,RPmag,BP-RP,RV,e_RV,Vbroad,GRVSmag,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,Teff,logg,__Fe_H_,Dist,A0,HIP,PS1,SDSS13,SKYM2,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,_2MASS,RAVE6,RAJ2000,DEJ2000,_q_2,_r_2,RA_ICRS_2,DE_ICRS_2,rgeo,b_rgeo,B_rgeo,rpgeo,b_rpgeo,B_rpgeo,Flag
km / s,K,mas / yr,,,deg,mas / yr,mas / yr,mas / yr,mas / yr,deg,,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,,,mag,,,mag,mag,km / s,km / s,km / s,mag,,,,,,,,,,,,K,log(cm.s**-2),,pc,mag,,,,,,,,,,,,,deg,deg,,,deg,deg,pc,pc,pc,pc,pc,pc,
float64,float64,float64,int32,float64,float64,float64,float32,float64,float32,float64,int64,float64,float64,float64,float32,float64,float64,float32,float64,float32,float64,float64,float32,float64,float32,float32,float64,float32,float32,float64,float64,float64,float32,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,int32,int64,int64,int32,str1,str15,str19,int32,str10,str1,str16,str1,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32
29.861,14941.699574067023,2.404,1,1.5205,339.89581711423,2.361,0.201,0.453,0.168,58.99547917199,2008384504995669248,0.0202,0.0197,0.3560,0.0241,1.404,-0.379,0.025,-1.352,0.024,1.086,13221.76503,3.482,15.384143,6055,14.81,15.883185,1.044e+04,18.25,14.701116,1.182070,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,9168.0,3.7061,-0.6147,4094.0940,2.4713,--,178793398958385263,--,--,--,URAT1-745392846,J223935.00+585943.5,13469566,N1C5026796,--,22393500+5859436,--,339.89582038348,58.99548518133,1,1.5205,339.89581711423,58.99547917199,2620.36230000,2461.53345000,2759.61401000,2605.56982000,2475.57690000,2754.27393000,10033
30.066,11849.49930976177,3.053,1,6.5222,339.69805696343,-2.180,0.201,-2.137,0.167,58.92918778013,2008382546490573824,0.0190,0.0193,0.4561,0.0228,6.305,-4.920,0.025,-3.942,0.023,1.046,13964.47166,3.916,15.324805,6086,13.27,15.877747,1.147e+04,11.21,14.599200,1.278546,--,--,--,--,0,0,0,1,0,0,0,0,0,1,0,--,--,--,--,--,--,178713396980935748,--,--,--,URAT1-745392015,--,--,--,--,22384755+5855451,--,339.69809933678,58.92920530089,1,6.5222,339.69805696343,58.92918778013,2077.95093000,1973.40735000,2173.33203000,2060.58984000,1965.45789000,2167.60474000,10033


CPU times: total: 2.56 s
Wall time: 3.1 s


In [21]:
search_arcmin = cluster.calculate_search_arcmin()
cluster.plot_search_region(display=None,pixels='800')
fits_path = f'{cluster.name}/{cluster.name}_extra10pc.fits'

fits_file = fits.open(fits_path)
image = fits_file[0]

wcs = WCS(image.header)

fig, ax2 = plt.subplots(subplot_kw={'projection': wcs}, figsize=(10, 8))
ax2.imshow(image.data, cmap='gray')
stars_in_region_coords = SkyCoord(ra=cluster.stars_in_region()['RA_ICRS_1'], dec=cluster.stars_in_region()['DE_ICRS_1'])
stars_in_region_pix_coords = wcs.world_to_pixel_values(stars_in_region_coords.ra, stars_in_region_coords.dec)

dias_members_coords = SkyCoord(ra=cluster.dias_members()['RAdeg']*u.deg, dec=cluster.dias_members()['DEdeg']*u.deg)
dias_members_pix_coords = wcs.world_to_pixel_values(dias_members_coords.ra, dias_members_coords.dec)
# Add a circle representing the cluster radius
circle_cluster = plt.Circle((image.data.shape[1] / 2, image.data.shape[0] / 2), radius=(image.shape[0]/2)*cluster.diameter/(2*search_arcmin),
                edgecolor='red', facecolor='none', ls='dashed', label=f'Cluster Diameter = {cluster.diameter}',linewidth=1)
circle_search_region = plt.Circle((image.data.shape[1] / 2, image.data.shape[0] / 2), radius=(image.shape[0]/2)*search_arcmin/search_arcmin,
            edgecolor='green', facecolor='none', ls='dashed', label=f'Search Region Diameter = {2*search_arcmin}',linewidth=1.5)
ax2.add_artist(circle_cluster)
ax2.add_artist(circle_search_region)
# Scatter Stars in region
scatter_stars_in_region = ax2.scatter(stars_in_region_pix_coords[0], stars_in_region_pix_coords[1], 
                                    s=1, color='grey',label=f'{len(cluster.stars_in_region())} Stars in the region')
# Scatter dias members
scatter_dias_members = ax2.scatter(dias_members_pix_coords[0], dias_members_pix_coords[1], s=12, color='grey',
                                edgecolor='lightgrey',label=f'{len(cluster.dias_members())} Dias Members')

# Scatter my members
my_members =find_cluster(cluster.stars_in_region(),refCluster=cluster.name)
my_members_coords = SkyCoord(ra=my_members['RA_ICRS_1'], dec=my_members['DE_ICRS_1'])
my_members_pix_coords = wcs.world_to_pixel_values(my_members_coords.ra, my_members_coords.dec)
scatter_my_members = ax2.scatter(my_members_pix_coords[0],my_members_pix_coords[1],
                                s=10, color='blue', marker='x', label=f'{len(my_members)} My Members')



# Read runaways
runaways_all = cluster.read_table('runaways') #changing this to runaways_all plots all the runaways, not just the >10000K ones
runaways_all_coords = SkyCoord(ra=runaways_all['RA_ICRS_1'], dec=runaways_all['DE_ICRS_1'])
runaways_all_pix_coords = wcs.world_to_pixel_values(runaways_all_coords.ra, runaways_all_coords.dec)



_ = search_arcmin.round(-1)/2 #round to the nearest 5 (by round first to nearest 10 and then divide by 2
scalebar_length = ((_.to(u.rad))*(cluster.distance.to(u.m)).to(u.pc)).round(2)
__ = cluster.distance.value/1000
add_scalebar(ax2, _, color="yellow", label=f"or {scalebar_length.value}pc (at dist {__:.2f}kpc)", size_vertical=0.5)
x_min, x_max = ax2.get_xlim()
y_min, y_max = ax2.get_ylim()
ax2.annotate(f"{_:.2f}", xy=(0.83*x_max,0.08*y_max), color='yellow', ha='center',fontsize=12,fontweight='bold')

ax2.set_xlabel('Right Ascension (degrees)')
ax2.set_ylabel('Declination (degrees)')
ax2.set_title(f"{cluster.name} with {config['Cluster']['search_extent']}pc search region")
ax2.grid(color='lightgrey', ls='dotted')


#######################################
runaway_00, runaway_apdp,runaway_apdm,runaway_amdp,runaway_amdm = [coord.SkyCoord(ra=runaways_all['RA_ICRS_1'],dec=runaways_all['DE_ICRS_1'], pm_ra_cosdec=runaways_all['rmRA'],pm_dec=runaways_all['rmDE'], frame='icrs',obstime=(Time('J2000')+1*u.Myr)),
                                                    coord.SkyCoord(ra=runaways_all['RA_ICRS_1'],dec=runaways_all['DE_ICRS_1'], pm_ra_cosdec=(runaways_all['rmRA']+runaways_all['e_rmRA']),pm_dec=runaways_all['rmDE']+runaways_all['e_rmDE'], frame='icrs',obstime=(Time('J2000')+1*u.Myr)),
                                                    coord.SkyCoord(ra=runaways_all['RA_ICRS_1'],dec=runaways_all['DE_ICRS_1'], pm_ra_cosdec=(runaways_all['rmRA']+runaways_all['e_rmRA']),pm_dec=runaways_all['rmDE']-runaways_all['e_rmDE'], frame='icrs',obstime=(Time('J2000')+1*u.Myr)),
                                                    coord.SkyCoord(ra=runaways_all['RA_ICRS_1'],dec=runaways_all['DE_ICRS_1'], pm_ra_cosdec=(runaways_all['rmRA']-runaways_all['e_rmRA']),pm_dec=runaways_all['rmDE']+runaways_all['e_rmDE'], frame='icrs',obstime=(Time('J2000')+1*u.Myr)),
                                                    coord.SkyCoord(ra=runaways_all['RA_ICRS_1'],dec=runaways_all['DE_ICRS_1'], pm_ra_cosdec=(runaways_all['rmRA']-runaways_all['e_rmRA']),pm_dec=runaways_all['rmDE']-runaways_all['e_rmDE'], frame='icrs',obstime=(Time('J2000')+1*u.Myr))]

earlier_runaway_00 = runaway_00.apply_space_motion(dt = -100_000*u.year)
earlier_runaway_apdp = runaway_apdp.apply_space_motion(dt = -100_000*u.year)
earlier_runaway_apdm = runaway_apdm.apply_space_motion(dt = -100_000*u.year)
earlier_runaway_amdp = runaway_amdp.apply_space_motion(dt = -100_000*u.year)
earlier_runaway_amdm = runaway_amdm.apply_space_motion(dt = -100_000*u.year)

earlier_runaway_00_px = wcs.world_to_pixel_values(earlier_runaway_00.ra, earlier_runaway_00.dec)
earlier_runaway_apdp_px = wcs.world_to_pixel_values(earlier_runaway_apdp.ra, earlier_runaway_apdp.dec)
earlier_runaway_apdm_px = wcs.world_to_pixel_values(earlier_runaway_apdm.ra, earlier_runaway_apdm.dec)
earlier_runaway_amdp_px = wcs.world_to_pixel_values(earlier_runaway_amdp.ra, earlier_runaway_amdp.dec)
earlier_runaway_amdm_px = wcs.world_to_pixel_values(earlier_runaway_amdm.ra, earlier_runaway_amdm.dec)
runaway_00_px = wcs.world_to_pixel_values(runaway_00.ra, runaway_00.dec)
# Calculate the displacement vectors
delta_x = earlier_runaway_apdp_px[0]-earlier_runaway_amdp_px[0]
delta_y = earlier_runaway_apdp_px[1]-earlier_runaway_apdm_px[1]

for pxx, pxy, dx, dy in zip(earlier_runaway_00_px[0], earlier_runaway_00_px[1], delta_x, delta_y):
    ellipse = patches.Ellipse(
        (pxx, pxy),
        width=1.5*dx,
        height=1.5*dy,
        fill=True,
        color='g',
        alpha=0.2
    )
    ax2.add_patch(ellipse)

for c in [earlier_runaway_apdp_px,earlier_runaway_apdm_px,earlier_runaway_amdp_px,earlier_runaway_amdm_px]:
    delta_x = c[0] - runaway_00_px[0]
    delta_y = c[1] - runaway_00_px[1]
    # Draw the vectors
    ax2.quiver(runaway_00_px[0], runaway_00_px[1], delta_x, delta_y, angles='xy', scale_units='xy', scale=1, color='limegreen', width=0.001)
############################
# Scatter runaways
scatter_runaways_all = ax2.scatter(runaways_all_pix_coords[0], runaways_all_pix_coords[1], s=60,picker=True,
                                c=runaways_all['Temp. Est'], cmap='bwr_r', norm=plt.Normalize(0, 30000), 
                                label=f'{len(runaways_all)} Runaway(s)')

# Function to toggle scatter plot visibility
def toggle_scatter(label, scatter):
    scatter.set_visible(not scatter.get_visible())
    plt.draw()

# Create check buttons
rax = plt.axes([0.0, 0.85, 0.22, 0.15])
check = CheckButtons(rax, ['Stars in the region', 'Dias Members', 'My Members','Runaway(s)'], [True, True, True, True])



# Connect the check buttons to the toggle function
check.on_clicked(lambda label: toggle_scatter(label, scatter_stars_in_region) if label == 'Stars in the region' else None)
check.on_clicked(lambda label: toggle_scatter(label, scatter_dias_members) if label == 'Dias Members' else None)
check.on_clicked(lambda label: toggle_scatter(label, scatter_runaways_all) if label == 'Runaway(s)' else None)
check.on_clicked(lambda label: toggle_scatter(label, scatter_my_members) if label == 'My Members' else None)

global annotations
annotations = []

def on_release(event):
    global annotations, table2
    for ann in annotations:
        ann.remove()
    annotations = []

    try:
        if table2:
            table2.remove()
    except:
        pass
    plt.draw() # for the release to remove functionality

def onpick3(event):
    global annotations, table2
    
    ind = event.ind
    for index in ind:
        index = ind[0]
        temp_est = runaways_all['Temp. Est'][index]
        # ann = ax2.annotate(f'{temp_est:,.0f} K', (0,0), xytext=(0, 0), textcoords='offset points', fontsize=12, color='black', ha='center', fontweight='bold')
        # ann.set_path_effects([pe.withStroke(linewidth=4, foreground='white')])
        # annotations.append(ann)
        
        # Add the table below the annotation
        table_data = [
            ['SourceID', f'{runaways_all[index]["Source"]}'],
            ['Temp. Est.', f'{runaways_all[index]["Temp. Est"]:,.0f} K'],
            ['Coord.', f'{runaways_all["RA_ICRS_1"][index]:.4f} {"+" if runaways_all["DE_ICRS_1"][index] >= 0 else ""}{runaways_all["DE_ICRS_1"][index]:.4f}'],
            ['Gmag', f'{runaways_all["Gmag"][index]}'],
            ['Dist.', f'{runaways_all["rgeo"][index]:.0f}'+'$\pm$'+f'{(runaways_all["rgeo"][index]-runaways_all["b_rgeo"][index]):.0f}']
            # Add more rows with actual values
        ]
        table_bbox = [0.0, 0.0, 0.45, 0.2]  # [left, bottom, width, height]
        table2 = ax2.table(cellText=table_data, cellLoc='right', loc='lower center', bbox=table_bbox)
        table2.auto_set_column_width(col=[0,1])
        for key, cell in table2._cells.items():
            cell.set_linewidth(0.5)  # Set the border width
            cell.set_edgecolor('lightgray')  # Set the border color
            # cell.get_text().set_weight('bold')
        
    plt.draw()

ax2.legend(loc='upper right')

colorbar = fig.colorbar(scatter_runaways_all,ax=ax2)
# Connect the pick_event to the onpick3 function
fig.canvas.mpl_connect('pick_event', onpick3)
fig.canvas.mpl_connect('button_release_event', on_release)

plt.show()
fits_file.close()


fits image exists in Berkeley_97 folder
Berkeley_97/Berkeley_97_stars_in_region.tsv exists in Berkeley_97 with 446 stars
Berkeley_97/Berkeley_97_stars_in_region.tsv exists in Berkeley_97 with 446 stars
Berkeley_97/Berkeley_97_stars_in_region.tsv exists in Berkeley_97 with 446 stars
Berkeley_97/Berkeley_97_stars_in_region.tsv exists in Berkeley_97 with 446 stars
mean(v_pec):  2.7685305586193776
std(v_pec):  2.3812339786561485
v_pec_max:  10.292515324099009


In [93]:
def plot_traceback(cluster):
    search_arcmin = cluster.calculate_search_arcmin()
    cluster.plot_search_region(display=None,pixels='800')
    fits_path = f'{cluster.name}/{cluster.name}_extra10pc.fits'

    fits_file = fits.open(fits_path)
    image = fits_file[0]

    wcs = WCS(image.header)

    fig, ax2 = plt.subplots(subplot_kw={'projection': wcs}, figsize=(10, 8))
    ax2.imshow(image.data, cmap='gray')
    stars_in_region_coords = SkyCoord(ra=cluster.stars_in_region()['RA_ICRS_1'], dec=cluster.stars_in_region()['DE_ICRS_1'])
    stars_in_region_pix_coords = wcs.world_to_pixel_values(stars_in_region_coords.ra, stars_in_region_coords.dec)

    dias_members_coords = SkyCoord(ra=cluster.dias_members()['RAdeg']*u.deg, dec=cluster.dias_members()['DEdeg']*u.deg)
    dias_members_pix_coords = wcs.world_to_pixel_values(dias_members_coords.ra, dias_members_coords.dec)
    # Add a circle representing the cluster radius
    circle_cluster = plt.Circle((image.data.shape[1] / 2, image.data.shape[0] / 2), radius=(image.shape[0]/2)*cluster.diameter/(2*search_arcmin),
                    edgecolor='red', facecolor='none', ls='dashed', label=f'Cluster Diameter = {cluster.diameter}',linewidth=1)
    circle_search_region = plt.Circle((image.data.shape[1] / 2, image.data.shape[0] / 2), radius=(image.shape[0]/2)*search_arcmin/search_arcmin,
                edgecolor='green', facecolor='none', ls='dashed', label=f'Search Region Diameter = {2*search_arcmin}',linewidth=1.5)
    ax2.add_artist(circle_cluster)
    ax2.add_artist(circle_search_region)
    # Scatter Stars in region
    scatter_stars_in_region, = ax2.plot(stars_in_region_pix_coords[0], stars_in_region_pix_coords[1], 
                                        '.',markersize=2, color='grey',label=f'{len(cluster.stars_in_region())} Stars in the region')
    # Scatter dias members
    scatter_dias_members, = ax2.plot(dias_members_pix_coords[0], dias_members_pix_coords[1], '.',markersize=10, color='grey',
                                    markeredgecolor='lightgrey',label=f'{len(cluster.dias_members())} Dias Members')
    # ax2.legend()
    # Scatter my members
    my_members =find_cluster(cluster.stars_in_region(),refCluster=cluster.name)
    my_members_coords = SkyCoord(ra=my_members['RA_ICRS_1'], dec=my_members['DE_ICRS_1'])
    my_members_pix_coords = wcs.world_to_pixel_values(my_members_coords.ra, my_members_coords.dec)
    scatter_my_members, = ax2.plot(my_members_pix_coords[0],my_members_pix_coords[1],
                                    '.',markersize=5, color='blue', label=f'{len(my_members)} My Members')



    # Read runaways
    runaways_all = cluster.read_table('runaways_all')[:5] #changing this to runaways_all plots all the runaways, not just the >10000K ones
    runaways_all_coords = SkyCoord(ra=runaways_all['RA_ICRS_1'], dec=runaways_all['DE_ICRS_1'])
    runaways_all_pix_coords = wcs.world_to_pixel_values(runaways_all_coords.ra, runaways_all_coords.dec)



    _ = search_arcmin.round(-1)/2 #round to the nearest 5 (by round first to nearest 10 and then divide by 2
    scalebar_length = ((_.to(u.rad))*(cluster.distance.to(u.m)).to(u.pc)).round(2)
    __ = cluster.distance.value/1000
    add_scalebar(ax2, _, color="yellow", label=f"or {scalebar_length.value}pc (at dist {__:.2f}kpc)", size_vertical=0.5)
    x_min, x_max = ax2.get_xlim()
    y_min, y_max = ax2.get_ylim()
    ax2.annotate(f"{_:.2f}", xy=(0.83*x_max,0.08*y_max), color='yellow', ha='center',fontsize=12,fontweight='bold')

    ax2.set_xlabel('Right Ascension (degrees)')
    ax2.set_ylabel('Declination (degrees)')
    ax2.set_title(f"{cluster.name} with {config['Cluster']['search_extent']}pc search region")
    ax2.grid(color='lightgrey', ls='dotted')


    #######################################
    runaway_00, runaway_apdp,runaway_apdm,runaway_amdp,runaway_amdm = [coord.SkyCoord(ra=runaways_all['RA_ICRS_1'],dec=runaways_all['DE_ICRS_1'], pm_ra_cosdec=runaways_all['rmRA'],pm_dec=runaways_all['rmDE'], frame='icrs',obstime=(Time('J2000')+1*u.Myr)),
                                                        coord.SkyCoord(ra=runaways_all['RA_ICRS_1'],dec=runaways_all['DE_ICRS_1'], pm_ra_cosdec=(runaways_all['rmRA']+runaways_all['e_rmRA']),pm_dec=runaways_all['rmDE']+runaways_all['e_rmDE'], frame='icrs',obstime=(Time('J2000')+1*u.Myr)),
                                                        coord.SkyCoord(ra=runaways_all['RA_ICRS_1'],dec=runaways_all['DE_ICRS_1'], pm_ra_cosdec=(runaways_all['rmRA']+runaways_all['e_rmRA']),pm_dec=runaways_all['rmDE']-runaways_all['e_rmDE'], frame='icrs',obstime=(Time('J2000')+1*u.Myr)),
                                                        coord.SkyCoord(ra=runaways_all['RA_ICRS_1'],dec=runaways_all['DE_ICRS_1'], pm_ra_cosdec=(runaways_all['rmRA']-runaways_all['e_rmRA']),pm_dec=runaways_all['rmDE']+runaways_all['e_rmDE'], frame='icrs',obstime=(Time('J2000')+1*u.Myr)),
                                                        coord.SkyCoord(ra=runaways_all['RA_ICRS_1'],dec=runaways_all['DE_ICRS_1'], pm_ra_cosdec=(runaways_all['rmRA']-runaways_all['e_rmRA']),pm_dec=runaways_all['rmDE']-runaways_all['e_rmDE'], frame='icrs',obstime=(Time('J2000')+1*u.Myr))]

    earlier_runaway_00 = runaway_00.apply_space_motion(dt = -100_000*u.year)
    earlier_runaway_apdp = runaway_apdp.apply_space_motion(dt = -100_000*u.year)
    earlier_runaway_apdm = runaway_apdm.apply_space_motion(dt = -100_000*u.year)
    earlier_runaway_amdp = runaway_amdp.apply_space_motion(dt = -100_000*u.year)
    earlier_runaway_amdm = runaway_amdm.apply_space_motion(dt = -100_000*u.year)

    earlier_runaway_00_px = wcs.world_to_pixel_values(earlier_runaway_00.ra, earlier_runaway_00.dec)
    earlier_runaway_apdp_px = wcs.world_to_pixel_values(earlier_runaway_apdp.ra, earlier_runaway_apdp.dec)
    earlier_runaway_apdm_px = wcs.world_to_pixel_values(earlier_runaway_apdm.ra, earlier_runaway_apdm.dec)
    earlier_runaway_amdp_px = wcs.world_to_pixel_values(earlier_runaway_amdp.ra, earlier_runaway_amdp.dec)
    earlier_runaway_amdm_px = wcs.world_to_pixel_values(earlier_runaway_amdm.ra, earlier_runaway_amdm.dec)
    runaway_00_px = wcs.world_to_pixel_values(runaway_00.ra, runaway_00.dec)
    # Calculate the displacement vectors
    delta_x = earlier_runaway_apdp_px[0]-earlier_runaway_amdp_px[0]
    delta_y = earlier_runaway_apdp_px[1]-earlier_runaway_apdm_px[1]

    for pxx, pxy, dx, dy in zip(earlier_runaway_00_px[0], earlier_runaway_00_px[1], delta_x, delta_y):
        ellipse = patches.Ellipse(
            (pxx, pxy),
            width=1.5*dx,
            height=1.5*dy,
            fill=True,
            color='g',
            alpha=0.2
        )
        ax2.add_patch(ellipse)

    for c in [earlier_runaway_apdp_px,earlier_runaway_apdm_px,earlier_runaway_amdp_px,earlier_runaway_amdm_px]:
        delta_x = c[0] - runaway_00_px[0]
        delta_y = c[1] - runaway_00_px[1]
        # Draw the vectors
        ax2.quiver(runaway_00_px[0], runaway_00_px[1], delta_x, delta_y, angles='xy', scale_units='xy', scale=1, color='limegreen', width=0.001)
    ############################
    # Scatter runaways
    scatter_runaways_all = ax2.scatter(runaways_all_pix_coords[0], runaways_all_pix_coords[1], s=60,picker=True,pickradius=20,
                                    c=runaways_all['Temp. Est'], cmap='bwr_r', norm=plt.Normalize(0, 30000), 
                                    label=f'{len(runaways_all)} Runaway(s)')


    global annotations
    annotations = []

    def on_release(event):
        global annotations, table2
        try:
            if table2:
                table2.remove()
        except:
            pass
        fig.canvas.draw() # for the release to remove functionality

    def onpick3(event):
        global annotations, table2
        
        ind = event.ind
        
        print(ind,type(ind),len(ind),event.artist)
        # print(result)
        import matplotlib
        if isinstance(event.artist, matplotlib.collections.PathCollection) and len(ind) == 1:
            for index in ind:
                index = ind[0]
                temp_est = runaways_all['Temp. Est'][index]
                # ann = ax2.annotate(f'{temp_est:,.0f} K', (0,0), xytext=(0, 0), textcoords='offset points', fontsize=12, color='black', ha='center', fontweight='bold')
                # ann.set_path_effects([pe.withStroke(linewidth=4, foreground='white')])
                # annotations.append(ann)
                
                # Add the table below the annotation
                table_data = [
                    ['SourceID', f'{runaways_all[index]["Source"]}'],
                    ['Temp. Est.', f'{runaways_all[index]["Temp. Est"]:,.0f} K'],
                    ['Coord.', f'{runaways_all["RA_ICRS_1"][index]:.4f} {"+" if runaways_all["DE_ICRS_1"][index] >= 0 else ""}{runaways_all["DE_ICRS_1"][index]:.4f}'],
                    ['Gmag', f'{runaways_all["Gmag"][index]}'],
                    ['Dist.', f'{runaways_all["rgeo"][index]:.0f}'+'$\pm$'+f'{(runaways_all["rgeo"][index]-runaways_all["b_rgeo"][index]):.0f}']
                    # Add more rows with actual values
                ]
                table_bbox = [0.0, 0.0, 0.45, 0.2]  # [left, bottom, width, height]
                table2 = ax2.table(cellText=table_data, cellLoc='right', loc='lower center', bbox=table_bbox)
                table2.auto_set_column_width(col=[0,1])
                for key, cell in table2._cells.items():
                    cell.set_linewidth(0.5)  # Set the border width
                    cell.set_edgecolor('lightgray')  # Set the border color
                    # cell.get_text().set_weight('bold')
            
            fig.canvas.draw()

    legend = ax2.legend(loc='upper right')
    scatter_stars_in_region_legend, scatter_dias_members_legend,scatter_my_members_legend = legend.get_lines()
    scatter_stars_in_region_legend.set_picker(True)
    scatter_stars_in_region_legend.set_pickradius(10)
    scatter_dias_members_legend.set_picker(True)
    scatter_dias_members_legend.set_pickradius(10)
    scatter_my_members_legend.set_picker(True)
    scatter_my_members_legend.set_pickradius(10)

    graphs = {}
    graphs[scatter_stars_in_region_legend] = scatter_stars_in_region
    graphs[scatter_dias_members_legend] = scatter_dias_members
    graphs[scatter_my_members_legend] = scatter_my_members


    colorbar = fig.colorbar(scatter_runaways_all,ax=ax2)
    # Connect the pick_event to the onpick3 function
    fig.canvas.mpl_connect('pick_event', onpick3)
    fig.canvas.mpl_connect('button_release_event', on_release)

    def on_pick(event):
        legend = event.artist
        isVisible = legend.get_visible()
        try:
            graphs[legend].set_visible(not isVisible)
            legend.set_visible(not isVisible)

        except:
            pass
        # fig.canvas.draw() # for the release to remove functionality

    plt.connect('pick_event', on_pick)


    fits_file.close()
cluster = Cluster('Ruprecht_170')
# runCode(cluster.name)
plot_traceback(cluster)

fits image exists in Ruprecht_170 folder
Ruprecht_170/Ruprecht_170_stars_in_region.tsv exists in Ruprecht_170 with 1906 stars
Ruprecht_170/Ruprecht_170_stars_in_region.tsv exists in Ruprecht_170 with 1906 stars
Ruprecht_170/Ruprecht_170_stars_in_region.tsv exists in Ruprecht_170 with 1906 stars
Ruprecht_170/Ruprecht_170_stars_in_region.tsv exists in Ruprecht_170 with 1906 stars
mean(v_pec):  8.46265827382422
std(v_pec):  7.98655346997783
v_pec_max:  27.042910140154405


[3 4] <class 'numpy.ndarray'> 2 <matplotlib.collections.PathCollection object at 0x0000019B12AD0D10>
[3 4] <class 'numpy.ndarray'> 2 <matplotlib.collections.PathCollection object at 0x0000019B12AD0D10>
[3 4] <class 'numpy.ndarray'> 2 <matplotlib.collections.PathCollection object at 0x0000019B12AD0D10>
[3] <class 'numpy.ndarray'> 1 <matplotlib.collections.PathCollection object at 0x0000019B12AD0D10>
[3] <class 'numpy.ndarray'> 1 <matplotlib.collections.PathCollection object at 0x0000019B12AD0D10>
[4] <class 'numpy.ndarray'> 1 <matplotlib.collections.PathCollection object at 0x0000019B12AD0D10>
[2] <class 'numpy.ndarray'> 1 <matplotlib.collections.PathCollection object at 0x0000019B12AD0D10>
[0] <class 'numpy.ndarray'> 1 <matplotlib.collections.PathCollection object at 0x0000019B12AD0D10>
[1] <class 'numpy.ndarray'> 1 <matplotlib.collections.PathCollection object at 0x0000019B12AD0D10>
[3] <class 'numpy.ndarray'> 1 <matplotlib.collections.PathCollection object at 0x0000019B12AD0D10>
[3 4

In [69]:
array2 = [False, True]
array1 = [True, False]

# Check if arrays have opposite patterns
result = array1 == array2[::-1]

print(result)


True


In [80]:
[1 ]

[1]

In [149]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(14, 8))

y = np.random.randint(0, 100, size=50)
x = np.random.choice(np.arange(len(y)), size=10)

line = ax.plot(y, '-', label='line')
dot = ax.plot(x, y[x], 'o', label='dot')

legend = plt.legend(loc='upper right')
line_legend, dot_legend = legend.get_lines()
line_legend.set_picker(True)
line_legend.set_pickradius(10)
dot_legend.set_picker(True)
dot_legend.set_pickradius(10)

graphs = {}
graphs[line_legend] = line
graphs[dot_legend] = dot


def on_pick(event):
    legend = event.artist
    isVisible = legend.get_visible()

    graphs[legend].set_visible(not isVisible)
    legend.set_visible(not isVisible)

    fig.canvas.draw()

plt.connect('pick_event', on_pick)
plt.show()

Traceback (most recent call last):
  File "C:\Users\sheth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "C:\Users\sheth\AppData\Local\Temp\ipykernel_4832\1144031456.py", line 173, in on_pick
    graphs[legend].set_visible(not isVisible)
    ~~~~~~^^^^^^^^
KeyError: <matplotlib.collections.PathCollection object at 0x000002CD4E4D0690>
Traceback (most recent call last):
  File "C:\Users\sheth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "C:\Users\sheth\AppData\Local\Temp\ipykernel_4832\1144031456.py", line 173, in on_pick
    graphs[legend].set_visible(not isVisible)
    ~~~~~~^^^^^^^^
KeyError: <matplotlib.collections.PathCollection object at 0x000002CD4E4D0690>
Traceback (most recent

In [48]:
legend.get_lines()

In [8]:
# from IPython.display import IFrame
# keyword = '130.25648735472 -48.06744090982'
# IFrame(rf"https://simbad.cds.unistra.fr/simbad/sim-basic?Ident={keyword}&submit=SIMBAD+search", 1400,600)

In [9]:
# coord = get_coord(cluster.read_table('runaways')[0])
# keyword = coord.to_string('hmsdms').replace('h','%20').replace('m','%20').replace('d','%20').replace('s','%20')
# IFrame(rf"https://aladin.cds.unistra.fr/AladinLite/?target={keyword}&fov={cluster.calculate_search_arcmin().value/60}&survey=CDS%2FP%2FDSS2%2Fcolor", 1400,600)